In [1]:
!pip install git+https://github.com/allenai/scibert.git
!pip install transformers
!pip install datasets

  Cloning https://github.com/allenai/scibert.git to /tmp/pip-req-build-penq6f5i
  Running command git clone -q https://github.com/allenai/scibert.git /tmp/pip-req-build-penq6f5i
  Cloning https://github.com/ibeltagy/allennlp (to revision fp16_and_others) to /tmp/pip-install-e72c_f28/allennlp_0f0c4d961bb348479247762ef716cf70
  Running command git clone -q https://github.com/ibeltagy/allennlp /tmp/pip-install-e72c_f28/allennlp_0f0c4d961bb348479247762ef716cf70
  Running command git checkout -b fp16_and_others --track origin/fp16_and_others
  Switched to a new branch 'fp16_and_others'
  Branch 'fp16_and_others' set up to track remote branch 'fp16_and_others' from 'origin'.
     |████████████████████████████████| 30.8 MB 1.6 MB/s 
     |████████████████████████████████| 125 kB 47.8 MB/s 
     |████████████████████████████████| 132 kB 53.7 MB/s 
     |████████████████████████████████| 5.8 MB 34.1 MB/s 
     |████████████████████████████████| 235 kB 54.4 MB/s 
     |██████████████████████████

In [ ]:
import zipfile
import numpy as np
import os

In [ ]:
# This is a direct-download version of the Kaggle dataset found at: https://www.kaggle.com/competitions/asap-sas/data
!wget 'https://drive.google.com/uc?export=download&id=1evWxj4M33SfSaw4nCXA8m71v9w8Zf1k2' -O dataset.zip

# Extract the dataset files to the "dataset" folder
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

--2022-04-21 22:02:18--  https://drive.google.com/uc?export=download&id=1evWxj4M33SfSaw4nCXA8m71v9w8Zf1k2
Resolving drive.google.com (drive.google.com)... 64.233.184.100, 64.233.184.102, 64.233.184.113, ...
Connecting to drive.google.com (drive.google.com)|64.233.184.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/07kutvrc6rfjb7d27r2mb5rm3r2mg5q2/1650578475000/09634012588375902450/*/1evWxj4M33SfSaw4nCXA8m71v9w8Zf1k2?e=download [following]
--2022-04-21 22:02:26--  https://doc-0o-24-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/07kutvrc6rfjb7d27r2mb5rm3r2mg5q2/1650578475000/09634012588375902450/*/1evWxj4M33SfSaw4nCXA8m71v9w8Zf1k2?e=download
Resolving doc-0o-24-docs.googleusercontent.com (doc-0o-24-docs.googleusercontent.com)... 172.253.120.132, 2a00:1450:400c:c01::84
Connecting to doc-0o-24-docs.googleusercontent.com (doc-0o-24-doc

In [ ]:
training_data_file_name = 'dataset/train.tsv'
test_data_texts_file_name = 'dataset/public_leaderboard_rel_2.tsv'
test_data_scores_file_name = 'dataset/public_leaderboard_solution.csv'

In [ ]:
training_data_tsv = np.genfromtxt(fname=training_data_file_name, delimiter="\t", skip_header=1, dtype='str')

training_data_texts = training_data_tsv[:,4]
training_data_essay_set = training_data_tsv[:,1].astype('int64')
training_data_scores = np.minimum(training_data_tsv[:,2].astype('int64'), training_data_tsv[:,3].astype('int64'))

print('Training Data Texts', training_data_texts.shape)
print('Training Data Scores', training_data_scores.shape)

test_data_texts_file = np.genfromtxt(fname=test_data_texts_file_name, delimiter="\t", skip_header=1, dtype='str')
test_data_scores_file = np.genfromtxt(fname=test_data_scores_file_name, delimiter=",", skip_header=1)

test_data_texts = test_data_texts_file[:,2]
test_data_essay_set = test_data_scores_file[:,1]
test_data_scores = test_data_scores_file[:,3]

print('Test Data Texts', test_data_texts.shape)
print('Test Data Scores', test_data_scores.shape)


train_matrix = np.column_stack((training_data_texts, training_data_essay_set.astype('str'), training_data_scores.astype('str')))
test_matrix = np.column_stack((test_data_texts, test_data_essay_set.astype('str'), test_data_scores.astype('str')))

try:
    os.remove('train.tsv')
except OSError:
    pass

try:
    os.remove('test.tsv')
except OSError:
    pass

np.savetxt('train.tsv', train_matrix, delimiter='\t', fmt='%s', header='text\tessay_set\tscore')
np.savetxt('test.tsv', test_matrix, delimiter='\t', fmt='%s', header='text\tessay_set\tscore')

Training Data Texts (17207,)
Training Data Scores (17207,)
Test Data Texts (5224,)
Test Data Scores (5224,)


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(training_data_texts, training_data_scores, test_size=.2)

In [ ]:
# get embeddings from SciBERT
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_data_texts), truncation=True, padding=True)


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# class ShortAnswerGradingDataset(Dataset):
#     def __init__(self, texts, essay_sets, scores):
#         self.scores = scores
#         self.essay_sets = essay_sets
#         self.texts = texts
#     def __len__(self):
#         return len(self.texts)
#     def __getitem__(self, idx):
#             score = self.scores[idx]
#             essay_set = self.scores[idx]
#             text = self.texts[idx]
#             sample = {"text": text, "score": score, "essay_set": essay_set}
#             return sample

class SASG_encodings(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# training_dataset = ShortAnswerGradingDataset(train_encodings, training_data_essay_set, training_data_scores)
# test_dataset = ShortAnswerGradingDataset(test_encodings, test_data_essay_set, test_data_scores)
train_dataset = SASG_encodings(train_encodings, train_labels)
val_dataset = SASG_encodings(val_encodings, val_labels)
test_dataset = SASG_encodings(test_encodings, test_data_scores)

In [ ]:
# np.unique(training_dataset.scores, return_counts=True)

# BERT trained on Scientific text: SciBERT

In [ ]:
# !git clone https://github.com/allenai/scibert.git

In [ ]:
#mount
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=13,  # batch size per device during training
    per_device_eval_batch_size=24,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=4)
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/results/checkpoint-1000")
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

# trainer.train()


# from transformers import *

# tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')


# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)
# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]

In [ ]:
!pip install datasets

     |████████████████████████████████| 325 kB 11.0 MB/s 
     |████████████████████████████████| 1.1 MB 21.2 MB/s 
     |████████████████████████████████| 136 kB 43.6 MB/s 
     |████████████████████████████████| 212 kB 47.5 MB/s 
     |████████████████████████████████| 271 kB 47.5 MB/s 
     |████████████████████████████████| 144 kB 47.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
  Attempting uninstall: responses
    Found existing installation: responses 0.20.0
    Uninstalling responses-0.20.0:
      Successfully uninstalled responses-0.20.0


In [ ]:
from sklearn.metrics import precision_recall_fscore_support

from datasets import load_metric
accuracy_score = load_metric("accuracy")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments


In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/results/checkpoint-1000")

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=13,  # batch size per device during training
    per_device_eval_batch_size=24,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

loading configuration file /content/drive/MyDrive/results/checkpoint-1000/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/results/checkpoint-1000",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
 

TypeError: ignored

In [ ]:
# torch.cuda.empty_cache()
!cp -r /content/results /content/drive/MyDrive/
!cp -r /content/logs /content/drive/MyDrive/

cp: cannot stat '/content/results': No such file or directory
cp: cannot stat '/content/logs': No such file or directory


In [ ]:
# sentence = “You look good today.” 
# # encode sentence (i.e. create input_ids, attention_mask)
# encoding = tokenizer(sentence)
# # make sure the keys of the "encoding" dict are on the same device as the model
# encoding = {k: v.to(trainer.args.device) for k, v in encoding.items()}
# # forward pass through the model
# with torch.no_grad():
#     outputs = trainer.model(**encoding)
# logits = outputs.logits 
# print("Predicted class index:", logits.argmax(-1))


In [ ]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# # To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
# # num_labels = len(model.config.id2label)
# # model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels)
# model = AutoModelForSequenceClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=4)

# labels = torch.tensor(1)
# loss = model(train_texts, labels=labels).loss
# round(loss.item(), 2)

In [ ]:
# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]


In [ ]:
# training_args = TrainingArguments(
#     output_dir="./results",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=5,
#     weight_decay=0.01,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_texts,
#     eval_dataset=val_texts,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# trainer.train()

In [ ]:
!pip install datasets

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/results/checkpoint-1000")

In [ ]:
# !pip install datasets==1.11.0

     |████████████████████████████████| 264 kB 4.9 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.5.1
    Uninstalling huggingface-hub-0.5.1:
      Successfully uninstalled huggingface-hub-0.5.1
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.18.0 requires huggingface-hub<1.0,>=0.1.0, but you have huggingface-hub 0.0.19 which is incompatible.


In [ ]:
# import datasets
# metric = datasets.load_metric('accuracy')
# for model_input in test_dataset:
# for i in range(len(test_data_texts)):
#   model_predictions = model(list(test_data_texts)[i])
#   metric.add_batch(predictions=model_predictions[i], references=test_data_scores[i])
# final_score = metric.compute()
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)


***** Running Prediction *****
  Num examples = 5224
  Batch size = 24


RuntimeError: ignored

In [ ]:
# model_predictions = model(test_data_texts)
test_dataset.texts

AttributeError: ignored

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
